# MACHINE LEARNING PROJECT

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import xgboost as xgb

# Importing Regressor Models
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

#Importing metrics
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc, roc_auc_score, roc_curve, confusion_matrix, f1_score
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, f1_score, precision_score, recall_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [ ]:
import tensorflow_addons as tfa

# Running Models

In [ ]:
data_cleaned = pd.read_csv('Final_Data_Modeling.csv')

## Splitting the data

We will split the data into train and test based on the seasons. We will keep the players of FIFA 22, i.e players in the season 2020/21

In [ ]:
test1= data_cleaned[data_cleaned['Season_fifa']=='2020/21']
train1= data_cleaned[data_cleaned['Season_fifa']!='2020/21']

In [ ]:
print(test1.shape, train1.shape)

(1522, 116) (7431, 116)


## Creating functions to execute the different models

In [ ]:
def run_model(X_train, y_train, epochs):
  #create model
  model = Sequential()

  #get number of columns in training data
  n_cols = X_train.shape[1]

  #add layers to model
  model.add(Dense(16, activation='relu', input_shape=(n_cols,)))
  # model.add(Dropout(0.4))
  model.add(BatchNormalization())

  model.add(Dense(64, activation='relu'))
  # model.add(Dropout(0.4))
  model.add(BatchNormalization())

  model.add(Dense(256, activation='relu'))
  # model.add(Dropout(0.4))
  model.add(BatchNormalization())

  model.add(Dense(1024, activation='relu'))
  # model.add(Dropout(0.4))
  model.add(BatchNormalization())

  model.add(Dense(2048, activation='relu'))
  # model.add(Dropout(0.4))
  model.add(BatchNormalization())

  model.add(Dense(1024, activation='relu'))
  # model.add(Dropout(0.4))
  model.add(BatchNormalization())

  model.add(Dense(256, activation='relu'))
  # model.add(Dropout(0.4))
  model.add(BatchNormalization())

  model.add(Dense(64, activation='relu'))
  # model.add(Dropout(0.4))
  model.add(BatchNormalization())

  model.add(Dense(16, activation='relu'))
  # model.add(Dropout(0.4))
  model.add(BatchNormalization())

  model.add(Dense(1, activation='linear'))

  #compile model using accuracy to measure model performance
  model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(), tfa.metrics.RSquare()])

  #set early stopping monitor so the model stops training when it won't improve anymore
  early_stopping_monitor = EarlyStopping(patience=3)

  # print (model.summary())

  #train model
  model.fit(X_train, y_train, epochs=epochs, validation_split=0.2, callbacks=early_stopping_monitor)

  return model

## Data 1 - All Players

### 1.a. Using All features

In [ ]:
%%time
train_1a= train1.copy(deep=True)
test_1a= test1.copy(deep=True)

X = train_1a.drop(['Season_fifa', 'overall', 'short_name'], axis=1)
y = train_1a['overall']

X_train, y_train = X, y

model = run_model(X_train, y_train, epochs=30)
test_1a['Overall_Predicted'] = model.predict(test_1a.drop(['Season_fifa', 'overall', 'short_name'], axis=1))
test_1a['Overall_Predicted'] = test_1a['Overall_Predicted'].apply(lambda x: round(x))

Epoch 1/30
186/186 [==============================] - 20s 87ms/step - loss: 5627.8296 - root_mean_squared_error: 75.0189 - r_square: -180.4956 - val_loss: 5519.2017 - val_root_mean_squared_error: 74.2913 - val_r_square: -192.2549
Epoch 2/30
186/186 [==============================] - 15s 78ms/step - loss: 5134.5244 - root_mean_squared_error: 71.6556 - r_square: -164.5795 - val_loss: 4613.3398 - val_root_mean_squared_error: 67.9216 - val_r_square: -160.5361
Epoch 3/30
186/186 [==============================] - 16s 86ms/step - loss: 4329.2461 - root_mean_squared_error: 65.7970 - r_square: -138.6167 - val_loss: 5088.3975 - val_root_mean_squared_error: 71.3330 - val_r_square: -177.1703
Epoch 4/30
186/186 [==============================] - 16s 85ms/step - loss: 3350.3286 - root_mean_squared_error: 57.8820 - r_square: -107.0493 - val_loss: 38257.1797 - val_root_mean_squared_error: 195.5944 - val_r_square: -1338.5757
Epoch 5/30
186/186 [==============================] - 14s 76ms/step - loss: 2

In [ ]:
test_1a[['short_name','overall', 'Overall_Predicted']].sort_values(by=['overall'], ascending=False).head(10)

,short_name,overall,Overall_Predicted
7431,L. Messi,93,68
7432,R. Lewandowski,92,70
7433,Cristiano Ronaldo,91,66
7434,Neymar Jr,91,71
7435,K. De Bruyne,91,71
7436,J. Oblak,91,68
7437,K. Mbappé,91,69
7438,M. Neuer,90,62
7439,M. ter Stegen,90,69
7440,H. Kane,90,70


### 1.b. Physical Attributes, Game attributes, wage, value

In [ ]:
%%time

cols = [0,1,5,6,7,10,11, 115]

train_1b= train1.copy(deep=True)
test_1a= test1.copy(deep=True)

X = train_1b.drop(train_1b.columns[cols], axis=1)
y = train_1b['overall']

X_train, y_train = X, y

model = run_model(X_train, y_train, epochs=30)
test_1a['Overall_Predicted']=model.predict(test_1a.drop(test_1a.columns[cols], axis=1))
test_1a['Overall_Predicted'] = test_1a['Overall_Predicted'].apply(lambda x: round(x))

Epoch 1/30
186/186 [==============================] - 19s 87ms/step - loss: 5633.2007 - root_mean_squared_error: 75.0546 - r_square: -180.6609 - val_loss: 5134.5894 - val_root_mean_squared_error: 71.6561 - val_r_square: -178.7877
Epoch 2/30
186/186 [==============================] - 14s 78ms/step - loss: 5146.6626 - root_mean_squared_error: 71.7402 - r_square: -164.9673 - val_loss: 4553.9893 - val_root_mean_squared_error: 67.4833 - val_r_square: -158.4580
Epoch 3/30
186/186 [==============================] - 14s 78ms/step - loss: 4343.6646 - root_mean_squared_error: 65.9065 - r_square: -139.0878 - val_loss: 3687.4009 - val_root_mean_squared_error: 60.7240 - val_r_square: -128.1144
Epoch 4/30
186/186 [==============================] - 15s 78ms/step - loss: 3363.9109 - root_mean_squared_error: 57.9992 - r_square: -107.4850 - val_loss: 2577.7175 - val_root_mean_squared_error: 50.7712 - val_r_square: -89.2588
Epoch 5/30
186/186 [==============================] - 15s 79ms/step - loss: 2369.

In [ ]:
test_1a[['short_name','overall', 'Overall_Predicted']].sort_values(by=['overall'], ascending=False).head(10)

,short_name,overall,Overall_Predicted
7431,L. Messi,93,92
7432,R. Lewandowski,92,95
7433,Cristiano Ronaldo,91,89
7434,Neymar Jr,91,96
7435,K. De Bruyne,91,96
7436,J. Oblak,91,94
7437,K. Mbappé,91,100
7438,M. Neuer,90,81
7439,M. ter Stegen,90,94
7440,H. Kane,90,96


### 1.c. Physical and Game Attributes

In [ ]:
%%time

cols = [0,1,5,6,7,8,9,10,11, 115]

train_1c= train1.copy(deep=True)
test_1a= test1.copy(deep=True)

X = train_1c.drop(train_1c.columns[cols], axis=1)
y = train_1c['overall']

X_train, y_train = X, y

model = run_model(X_train, y_train, epochs=30)
test_1a['Overall_Predicted']=model.predict(test_1a.drop(test_1a.columns[cols], axis=1))
test_1a['Overall_Predicted'] = test_1a['Overall_Predicted'].apply(lambda x: round(x))

Epoch 1/30
186/186 [==============================] - 19s 86ms/step - loss: 5635.2466 - root_mean_squared_error: 75.0683 - r_square: -180.7388 - val_loss: 5271.7227 - val_root_mean_squared_error: 72.6066 - val_r_square: -183.5894
Epoch 2/30
186/186 [==============================] - 14s 77ms/step - loss: 5131.2451 - root_mean_squared_error: 71.6327 - r_square: -164.4809 - val_loss: 4978.2207 - val_root_mean_squared_error: 70.5565 - val_r_square: -173.3125
Epoch 3/30
186/186 [==============================] - 14s 76ms/step - loss: 4321.8701 - root_mean_squared_error: 65.7409 - r_square: -138.3789 - val_loss: 3812.2815 - val_root_mean_squared_error: 61.7437 - val_r_square: -132.4871
Epoch 4/30
186/186 [==============================] - 14s 77ms/step - loss: 3343.5063 - root_mean_squared_error: 57.8231 - r_square: -106.8246 - val_loss: 2767.8291 - val_root_mean_squared_error: 52.6102 - val_r_square: -95.9156
Epoch 5/30
186/186 [==============================] - 14s 77ms/step - loss: 2353.

In [ ]:
test_1a[['short_name','overall', 'Overall_Predicted']].sort_values(by=['overall'], ascending=False).head(10)

,short_name,overall,Overall_Predicted
7431,L. Messi,93,74
7432,R. Lewandowski,92,73
7433,Cristiano Ronaldo,91,73
7434,Neymar Jr,91,73
7435,K. De Bruyne,91,73
7436,J. Oblak,91,72
7437,K. Mbappé,91,72
7438,M. Neuer,90,73
7439,M. ter Stegen,90,72
7440,H. Kane,90,72


## Data 2 - Players who have played more than 10 games

In [ ]:
df2 = data_cleaned[data_cleaned['GP']>10]

In [ ]:
df2.shape

(7020, 116)

In [ ]:
test2= df2[df2['Season_fifa']=='2020/21']
train2= df2[df2['Season_fifa']!='2020/21']

In [ ]:
print(test2.shape, train2.shape)

(1236, 116) (5784, 116)


### 2.a. All data

In [ ]:
%%time

train_2a= train2.copy(deep=True)
test_2a= test2.copy(deep=True)

X = train_2a.drop(['Season_fifa', 'overall', 'short_name'], axis=1)
y = train_2a['overall']

X_train, y_train = X, y

model = run_model(X_train, y_train, epochs=30)

Epoch 1/30
145/145 [==============================] - 16s 85ms/step - loss: 5868.9922 - root_mean_squared_error: 76.6093 - r_square: -264.3698 - val_loss: 5669.3550 - val_root_mean_squared_error: 75.2951 - val_r_square: -317.6594
Epoch 2/30
145/145 [==============================] - 12s 84ms/step - loss: 5566.7017 - root_mean_squared_error: 74.6103 - r_square: -250.7164 - val_loss: 5102.0122 - val_root_mean_squared_error: 71.4284 - val_r_square: -285.7705
Epoch 3/30
145/145 [==============================] - 12s 86ms/step - loss: 5038.7188 - root_mean_squared_error: 70.9839 - r_square: -226.8508 - val_loss: 4352.7539 - val_root_mean_squared_error: 65.9754 - val_r_square: -243.6567
Epoch 4/30
145/145 [==============================] - 13s 88ms/step - loss: 4340.1162 - root_mean_squared_error: 65.8796 - r_square: -195.2639 - val_loss: 4306.0093 - val_root_mean_squared_error: 65.6202 - val_r_square: -241.0293
Epoch 5/30
145/145 [==============================] - 13s 90ms/step - loss: 3546

In [ ]:
test_2a['Overall_Predicted'] = model.predict(test_2a.drop(['Season_fifa', 'overall', 'short_name'], axis=1))
test_2a['Overall_Predicted'] = test_2a['Overall_Predicted'].apply(lambda x: round(x))

test_2a[['short_name','overall', 'Overall_Predicted']].sort_values(by=['overall'], ascending=False).head(10)

,short_name,overall,Overall_Predicted
7431,L. Messi,93,78
7432,R. Lewandowski,92,71
7433,Cristiano Ronaldo,91,79
7434,Neymar Jr,91,69
7435,K. De Bruyne,91,72
7436,J. Oblak,91,65
7437,K. Mbappé,91,54
7438,M. Neuer,90,72
7439,M. ter Stegen,90,72
7440,H. Kane,90,68


### 2.b Physical Attributes, Game Attributes, Values and Wages

In [ ]:
%%time

cols = [0,1,5,6,7,10,11, 115]

train_2b= train2.copy(deep=True)
test_2a= test2.copy(deep=True)

X = train_2b.drop(train_2b.columns[cols], axis=1)
y = train_2b['overall']

X_train, y_train = X, y

model = run_model(X_train, y_train, epochs=30)
test_2a['Overall_Predicted']=model.predict(test_2a.drop(test_2a.columns[cols], axis=1))
test_2a['Overall_Predicted'] = test_2a['Overall_Predicted'].apply(lambda x: round(x))

Epoch 1/30
145/145 [==============================] - 15s 85ms/step - loss: 5854.9233 - root_mean_squared_error: 76.5175 - r_square: -263.7182 - val_loss: 5547.0493 - val_root_mean_squared_error: 74.4785 - val_r_square: -310.7849
Epoch 2/30
145/145 [==============================] - 12s 81ms/step - loss: 5529.7812 - root_mean_squared_error: 74.3625 - r_square: -249.0518 - val_loss: 5115.6860 - val_root_mean_squared_error: 71.5240 - val_r_square: -286.5391
Epoch 3/30
145/145 [==============================] - 12s 84ms/step - loss: 4989.0054 - root_mean_squared_error: 70.6329 - r_square: -224.5940 - val_loss: 4351.9121 - val_root_mean_squared_error: 65.9690 - val_r_square: -243.6094
Epoch 4/30
145/145 [==============================] - 13s 87ms/step - loss: 4285.1641 - root_mean_squared_error: 65.4612 - r_square: -192.7561 - val_loss: 3718.9875 - val_root_mean_squared_error: 60.9835 - val_r_square: -208.0344
Epoch 5/30
145/145 [==============================] - 12s 85ms/step - loss: 3493

In [ ]:
test_2a[['short_name','overall', 'Overall_Predicted']].sort_values(by=['overall'], ascending=False).head(10)

,short_name,overall,Overall_Predicted
7431,L. Messi,93,91
7432,R. Lewandowski,92,95
7433,Cristiano Ronaldo,91,87
7434,Neymar Jr,91,96
7435,K. De Bruyne,91,96
7436,J. Oblak,91,94
7437,K. Mbappé,91,101
7438,M. Neuer,90,78
7439,M. ter Stegen,90,93
7440,H. Kane,90,96


### 2.c Physical and Game Attributes

In [ ]:
%%time

cols = [0,1,5,6,7,8,9,10,11, 115]

train_2c= train2.copy(deep=True)
test_2a= test2.copy(deep=True)

X = train_2c.drop(train_2c.columns[cols], axis=1)
y = train_2c['overall']

X_train, y_train = X, y

model = run_model(X_train, y_train, epochs=30)
test_2a['Overall_Predicted']=model.predict(test_2a.drop(test_2a.columns[cols], axis=1))
test_2a['Overall_Predicted'] = test_2a['Overall_Predicted'].apply(lambda x: round(x))

Epoch 1/30
145/145 [==============================] - 16s 89ms/step - loss: 5875.6152 - root_mean_squared_error: 76.6526 - r_square: -264.6745 - val_loss: 5609.5923 - val_root_mean_squared_error: 74.8972 - val_r_square: -314.3003
Epoch 2/30
145/145 [==============================] - 13s 87ms/step - loss: 5562.5854 - root_mean_squared_error: 74.5827 - r_square: -250.5302 - val_loss: 5197.8394 - val_root_mean_squared_error: 72.0960 - val_r_square: -291.1567
Epoch 3/30
145/145 [==============================] - 11s 74ms/step - loss: 5028.3750 - root_mean_squared_error: 70.9110 - r_square: -226.3742 - val_loss: 4880.1846 - val_root_mean_squared_error: 69.8583 - val_r_square: -273.3022
Epoch 4/30
145/145 [==============================] - 13s 88ms/step - loss: 4327.2314 - root_mean_squared_error: 65.7817 - r_square: -194.6812 - val_loss: 3815.2839 - val_root_mean_squared_error: 61.7680 - val_r_square: -213.4470
Epoch 5/30
145/145 [==============================] - 12s 84ms/step - loss: 3534

In [ ]:
test_2a[['short_name','overall', 'Overall_Predicted']].sort_values(by=['overall'], ascending=False).head(10)

,short_name,overall,Overall_Predicted
7431,L. Messi,93,100
7432,R. Lewandowski,92,88
7433,Cristiano Ronaldo,91,92
7434,Neymar Jr,91,90
7435,K. De Bruyne,91,89
7436,J. Oblak,91,93
7437,K. Mbappé,91,92
7438,M. Neuer,90,91
7439,M. ter Stegen,90,87
7440,H. Kane,90,90
